# Initial exploration: how well does the orig. authors' FFF-BERT perform

Answer: Not impressive. Slower on M2 mac than an equiv sized vanilla BERT model

# 🔸 reference (BERT)

https://huggingface.co/bert-base-uncased

https://huggingface.co/bert-base-multilingual-cased  <-- use this for fairness, similar size to our model

In [1]:
from transformers import pipeline
# unmasker = pipeline('fill-mask', model='bert-base-uncased')  # 110M params
unmasker = pipeline('fill-mask', model='bert-base-multilingual-cased')  # 179M params, 714MB


/Users/pi/code/m2/fff/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 625/625 [00:00<00:00, 1.84MB/s]
model.safetensors: 100%|██████████| 714M/714M [01:28<00:00, 8.07MB/s] 
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (ini

In [2]:
%timeit unmasker("The capital of France is [MASK]!")

62.6 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


https://huggingface.co/pbelcak/UltraFastBERT-1x11-long

# 🔸 Our model

https://huggingface.co/pbelcak/UltraFastBERT-1x11-long

(189M params)

In [3]:
import cramming

In [4]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("pbelcak/UltraFastBERT-1x11-long")
model = AutoModelForMaskedLM.from_pretrained("pbelcak/UltraFastBERT-1x11-long")


In [5]:
text = "The capital of France is <mask>!"
# text = "The cat sat on <mask> mat."
encoded_input = tokenizer(text, return_tensors='pt')

In [6]:
%timeit output = model(**encoded_input)

77.6 ms ± 2.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
output = model(**encoded_input)

mask_token_index = (encoded_input.input_ids[0] == tokenizer.mask_token_id).nonzero(as_tuple=True)[0]
logits = output['outputs']
top5_indices = torch.topk(logits[mask_token_index], k=5).indices
token_winners = [tokenizer.decode(u) for u in top5_indices]
print('Text:', text)
print(f'Most likely token prediction: {token_winners}')

Text: The capital of France is <mask>!
Most likely token prediction: ['paris france french here germany']
